# Decision Tree from Scratch 

## Data

In [1]:
import math
import pandas as pd
import numpy as np

# Lenses data https://archive.ics.uci.edu/ml/machine-learning-databases/lenses/lenses.data
# Directly add here for easiness

data = [[1, 1, 1, 1, 1, 3],
[2, 1, 1, 1, 2, 2],
[3, 1, 1, 2, 1, 3],
[4, 1, 1, 2, 2, 1],
[5, 1, 2, 1, 1, 3],
[6, 1, 2, 1, 2, 2],
[7, 1, 2, 2, 1, 3],
[8, 1, 2, 2, 2, 1],
[9, 2, 1, 1, 1, 3],
[10, 2, 1, 1, 2, 2],
[11, 2, 1, 2, 1, 3],
[12, 2, 1, 2, 2, 1],
[13, 2, 2, 1, 1, 3],
[14, 2, 2, 1, 2, 2],
[15, 2, 2, 2, 1, 3],
[16, 2, 2, 2, 2, 3],
[17, 3, 1, 1, 1, 3],
[18, 3, 1, 1, 2, 3],
[19, 3, 1, 2, 1, 3],
[20, 3, 1, 2, 2, 1],
[21, 3, 2, 1, 1, 3],
[22, 3, 2, 1, 2, 2],
[23, 3, 2, 2, 1, 3],
[24, 3, 2, 2, 2, 3]]

df = pd.DataFrame(data, columns=['num','age','prescription','astigmatic','tear_rate','lens'])

In [91]:
class decision_tree:
    def __init__(self):
        self.split_value = 0
        self.split_attr = -1
        self.max_gain = 0.1
        self.left = None
        self.right = None
        self.depth = 0
        self.distribution = []

def entropy(data, target_attr):
    # Calculates the entropy of the given data set for the target attribute.
    
    # get unique values of target variable
    target_vals = {row[target_attr] for row in data}
    
    n = len(data)
    
    # counter for each target values
    p = {x: 0 for x in target_vals}
    
    data_entropy = 0
    
    # count each target values in the data
    for val in target_vals:
        for row in data:
            if row[1] == val: p[val] += 1
    
    # calculate entropy using the counter
    for x in p.values():
        if x == 0: print(data)
        data_entropy -= (x/n)*np.log(x/n)
        
    return data_entropy

def gain(data, attr, target_attr):
    # Calculates the information gain (reduction in entropy)
    # that would result by splitting the data on the chosen attribute (attr).
    # Write your code here
    attr_vals = {row[attr] for row in data}
    attr_count = {x: 0 for x in attr_vals}

    n = len(data)
    subdata = [[row[attr], row[target_attr]] for row in data]
    data_entropy = entropy(subdata, 1)
    
    subdata = np.array(subdata)
    sub_category = set(subdata[:, 0])
    target_category = set(subdata[:, 1])
    n = len(subdata)

    gain = data_entropy

    p = {x: {y: 0 for y in target_category } for x in sub_category}

    for i in p.keys():
        gain -= ((subdata[:, 0] == i).sum() / n * entropy(subdata[subdata[:, 0] == i], 1))
        for j in target_category:
            p[i][j] = (subdata[subdata[:, 0] == i][:, 1] == j).sum()


    return gain

def growTree(node, data,target_attr):
    # which attribute and value is best to split
    for i in range(len(data)): # go over each instance to split to left/right
        for j in range(len(data[0])): # in each instance, go over each attribute
            if gain(data,j,target_attr) > node.max_gain and j != target_attr:
                node.split_value = data[i][j]
                node.split_attr = j
                node.max_gain = gain(data,j,target_attr)

    if node.split_attr != -1:           
        
        print("=" * node.depth + "v" + str(node.split_attr) + ">" + str(node.split_value))
                
        # now split data
        left_data, right_data = [], []
        for i in range(len(data)):
            if data[i][node.split_attr] > node.split_value:
                right_data.append(data[i])
            else:
                left_data.append(data[i])
        node.distribution = {i:[row[-1] for row in data].count(i) for i in set([row[-1] for row in data])}
    if node.split_attr == -1 or len(data) <2 or len(right_data) <1 or len(left_data) <1:
        # no more splitting
        node.leaf = True
        node.distribution = {i:[row[-1] for row in data].count(i) for i in set([row[-1] for row in data])}
        # max(dic, key=dic.get) returns the key that has max value in a dict namely dic
        print("=" * node.depth + "leaf: classify to class=" + str(max(node.distribution, key=node.distribution.get)) + " with class distribution " + "[" + str(node.distribution) + "]")
        return node

    node.left = decision_tree()
    node.left.depth = node.depth + 1
    node.right = decision_tree()
    node.right.depth = node.depth + 1
    growTree(node.left, left_data, target_attr)
    growTree(node.right, right_data, target_attr)

def classify(node, X):
    # classify test instances in X given a tree
    # Write your code here
    pred = np.array([])
    
    if node.split_attr == -1:
        pred = np.append(pred, np.array(max(node.distribution, key=node.distribution.get)))
        return pred
    else:
        for row in X:
            if row[node.split_attr] > node.split_value:
                pred = np.append(pred, np.array(classify(node.right, [row])))
            else:
                pred = np.append(pred, classify(node.left, [row]))
    
    return pred

In [50]:
tree = decision_tree()
clean_data = [row[1:] for row in data]
growTree(tree,clean_data, 4)

v3>1
=leaf: classify to class=3 with class distribution [{3: 12}]
=v2>1
==v0>1
===leaf: classify to class=2 with class distribution [{2: 2}]
===v0>2
====leaf: classify to class=2 with class distribution [{2: 2}]
====v1>1
=====leaf: classify to class=3 with class distribution [{3: 1}]
=====leaf: classify to class=2 with class distribution [{2: 1}]
==v1>1
===leaf: classify to class=1 with class distribution [{1: 3}]
===v0>1
====leaf: classify to class=1 with class distribution [{1: 1}]
====leaf: classify to class=3 with class distribution [{3: 2}]


In [93]:
pred = classify(tree, clean_data)
y = [row[-1] for row in data]
print("Accuracy: %f\n" % (np.mean(y == pred) * 100))

Accuracy: 100.000000

